**Table of contents**<a id='toc0_'></a>    
- 1. [DateTime](#toc1_)    
  - 1.1. [Creating DataTime](#toc1_1_)    
  - 1.2. [Filtering DateTimeIndex](#toc1_2_)    
  - 1.3. [Working with TimeZones](#toc1_3_)    
  - 1.4. [Operation on DataIndexTime](#toc1_4_)    
  - 1.5. [Reasampling and upsampling](#toc1_5_)    
  - 1.6. [Strivng diplays](#toc1_6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=2
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[DateTime](#toc0_)

## 1.1. <a id='toc1_1_'></a>[Creating DataTime](#toc0_)

In [ ]:
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"

In [ ]:
"D - day freqyency"
#pd.date_range()
daily_index = pd.date_range("2020.02.28",periods=4,freq="D")
daily_index

In [ ]:
weekly_index = pd.date_range("2020.01.05", "2020.01.31", freq="W-SUN")
weekly_index

In [ ]:
"Implementing index in DataFrame"
pd.DataFrame(data=[21,15,33,34], columns=["odwiedzający"], index=weekly_index)

In [ ]:
msft = pd.read_csv("D:\Repozytorium\Kody_Python\Python_ksiazka_przyklady\csv\MSFT.csv")
msft

In [ ]:
"Range Index"
msft.info()

In [ ]:
"Conreting column to datetime object"
#pd.to_datetime
msft.loc[:,["Date"]] = pd.to_datetime(msft["Date"])
msft.dtypes

In [ ]:
"Sorting index before starting"
#sort_index()
msft.sort_index()

In [ ]:
"Different approach to set Datetime"
#parse_dates, adtype()
msft = pd.read_csv("D:\Repozytorium\Kody_Python\Python_ksiazka_przyklady\csv\MSFT.csv",index_col="Date", parse_dates=["Date"])
msft.loc[:,"Volume"] = msft["Volume"].astype("float")

In [ ]:
"Getting acces to different datas can be Year, month,day"
msft.index.date

## 1.2. <a id='toc1_2_'></a>[Filtering DateTimeIndex](#toc0_)

In [ ]:
"Checking 2019 and Adj Close data"
msft.loc["2019","Adj Close"]

In [ ]:
"Range from 2019.06 to 2020.05"
msft.loc["2019/06":"2020/05","Adj Close"].plot()

## 1.3. <a id='toc1_3_'></a>[Working with TimeZones](#toc0_)

In [ ]:
"Inserting data with time information"
#pd.DateOffset()
msft_close = msft.loc[:,["Adj Close"]].copy()
msft_close.index = msft_close.index + pd.DateOffset(hours=16)
msft_close

In [ ]:
'Data setting with and time changing DST (winter summer)'
#tz_localize()
msft_close = msft_close.tz_localize("America/New_York")
msft_close

In [ ]:
"With DST 21:00"
#tz_convert
msft_close = msft_close.tz_convert("UTC")
msft_close.loc["2020-01-02","Adj Close"]

In [ ]:
"With DST 20:00"
msft_close.loc["2020-05-01","Adj Close"]

## 1.4. <a id='toc1_4_'></a>[Operation on DataIndexTime](#toc0_)

In [ ]:
msft_close.head()

In [ ]:
"Shifting 1 value down"
#shift()
msft_close.shift(1).head()

In [ ]:
"Assainng"
returns = np.log(msft_close/msft_close.shift(1))
returns

In [ ]:
"Replace can't be used cause its for values, need to use rename"
#rename
returns = returns.rename(columns={"Adj Close":"Stopa Zwrotu"})

In [ ]:
returns

In [ ]:
returns.plot.hist()

In [ ]:
"Simpler method (percentage change)"
#pct_change()
simple_rets = msft_close.pct_change()
simple_rets = simple_rets.rename(columns={"Adj Close":"Prosta Stopa Zwrotu"})
simple_rets

### Correlation

In [ ]:
"More data to merge"
parts = []
for file in ["AAPL", "AMZN", "GOOGL", "MSFT"]:
    #usecols -> reading only those columns
    adj_close = pd.read_csv(f"D:\Repozytorium\Kody_Python\Python_ksiazka_przyklady\csv\{file}.csv",index_col="Date", parse_dates=["Date"], usecols=["Date", "Adj Close"])
    #Changing column name for every file
    adj_close = adj_close.rename(columns={"Adj Close":file})
    #adding to parts list to summarize all tables
    parts.append(adj_close)
parts

In [ ]:
"Using concat() formula for merging data"
#conat()
adj_close = pd.concat(parts,axis=1)
adj_close

In [ ]:
"Deleting all data when NA value is present"
#dropna()
adj_close = adj_close.dropna()
adj_close

In [ ]:
"Chooding data from 2019-06 to 2020-05"
adj_close_sample = adj_close.loc["2019-06":"2020-05",:]
adj_close_sample

In [ ]:
"Insertin formula starting from 100 to check prices changes"
rebasec_prices = adj_close_sample/adj_close_sample.iloc[0,:]*100

In [ ]:
rebasec_prices.plot()

In [ ]:
"Creating correlation"
returns = np.log(adj_close/adj_close.shift(1))
returns.corr()

In [ ]:
import plotly.express as px
fig = px.imshow(returns.corr(), x=adj_close.columns, y=adj_close.columns, color_continuous_scale=list(reversed(px.colors.sequential.RdBu)),zmin=-1,zmax=1)
fig.show()

## 1.5. <a id='toc1_5_'></a>[Reasampling and upsampling](#toc0_)

In [ ]:
"besides last you can use diffferent method like sum, mean, ohlc)"
#resample()
end_of_month = adj_close.resample("M").last()
end_of_month

In [ ]:
#asfreq()
end_of_month.resample("D").asfreq().head()

In [ ]:
"Asfeq is an information to not give and transformations that's why i woud be NAN, if we use ffi;; then it wiel be last known"
#ffill()
end_of_month.resample("W-FRI").ffill().head()

## 1.6. <a id='toc1_6_'></a>[Strivng diplays](#toc0_)

In [ ]:
msft19 = msft.loc["2019",["Adj Close"]].copy()
msft19

In [ ]:
"Średnia z 25 poprzednich dni"
#rolling()
msft19.loc[:,"średnia z 25 dni"] = msft19["Adj Close"].rolling(25).mean()
msft19.plot()